<a href="https://colab.research.google.com/github/bhargav-joshi/Baby-Names-Predictor/blob/master/Baby_Names_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_csv("NationalNames.csv")

In [5]:
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [6]:
data.shape

(1825433, 5)

In [7]:
data['Name'].nunique()

93889

In [8]:
le = LabelEncoder()
data['Gender']  = le.fit_transform(data['Gender'])

In [9]:
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,0,7065
1,2,Anna,1880,0,2604
2,3,Emma,1880,0,2003
3,4,Elizabeth,1880,0,1939
4,5,Minnie,1880,0,1746


In [10]:
df = data.groupby('Name').mean()[['Gender']].reset_index()

In [11]:
df['Gender'] =  df['Gender'].astype('int')

In [12]:
df.head()

,Name,Gender
0,Aaban,1
1,Aabha,0
2,Aabid,1
3,Aabriella,0
4,Aadam,1


In [13]:
import string

In [14]:
letters = list(string.ascii_lowercase)

In [15]:
vocab = dict(zip(letters, range(1, 27)))

In [16]:
rev_dic = dict(zip( range(1, 27), letters))

In [17]:
name = "Aabriella".lower()

In [18]:
seq = [vocab[i] for i in name]

In [19]:
seq

[1, 1, 2, 18, 9, 5, 12, 12, 1]

In [20]:
X = []
for name in df['Name'].values:
    name = name.lower()
    seq = [vocab[i] for i in name]
    X.append(seq)

In [21]:
y = df.Gender.values

In [22]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [23]:
X = pad_sequences(X, maxlen=10, padding='pre')

In [24]:
X.shape

(93889, 10)

In [25]:
y.shape

(93889,)

In [26]:
from keras.layers import *
from keras.models import Model

In [27]:
len(vocab)

26

In [28]:
inp = Input(shape=(10,))
emb = Embedding(input_dim=27, output_dim = 5)(inp)
rnn = SimpleRNN(units=32)(emb)
out = Dense(units=1, activation='sigmoid')(rnn)

In [29]:
model = Model(inputs = inp, outputs = out)

In [30]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 5)             135       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1216      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,384
Trainable params: 1,384
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam', loss="binary_crossentropy", metrics = ['accuracy'])

In [32]:
hist = model.fit(X, y, epochs=10, batch_size=1024, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 75111 samples, validate on 18778 samples
Epoch 1/10
75111/75111 [==============================] - 1s 19us/step - loss: 0.6116 - accuracy: 0.6812 - val_loss: 0.5481 - val_accuracy: 0.7161
Epoch 2/10
75111/75111 [==============================] - 1s 12us/step - loss: 0.4989 - accuracy: 0.7457 - val_loss: 0.4408 - val_accuracy: 0.7959
Epoch 3/10
75111/75111 [==============================] - 1s 12us/step - loss: 0.4265 - accuracy: 0.7970 - val_loss: 0.3999 - val_accuracy: 0.8164
Epoch 4/10
75111/75111 [==============================] - 1s 12us/step - loss: 0.3987 - accuracy: 0.8148 - val_loss: 0.3926 - val_accuracy: 0.8231
Epoch 5/10
75111/75111 [==============================] - 1s 13us/step - loss: 0.3927 - accuracy: 0.8183 - val_loss: 0.3919 - val_accuracy: 0.8218
Epoch 6/10
75111/75111 [==============================] - 1s 12us/step - loss: 0.3903 - accuracy: 0.8204 - val_loss: 0.3913 - val_accuracy: 0.8237
Epoch 7/10
75111/75111 [==============================] - 1s 12us/st

In [38]:
name = "Bhargav".lower()

In [39]:
seq = [vocab[i] for i in name]

In [40]:
x_test = pad_sequences([seq], maxlen=10)

In [41]:
x_test

array([[ 0,  0,  0,  2,  8,  1, 18,  7,  1, 22]], dtype=int32)

In [42]:
model.predict(x_test)

array([[0.8844215]], dtype=float32)

In [44]:
# 0.1-0.5 => girl
# 0.6-0.9 => boy